# Example use of the Sonic module Nodosus.jl

In [1]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [2]:
import Dates, JuliaDB, Plots, Random

In [21]:
include("../src/Linckii.jl")
include("../src/LinckiiSecrets.jl")
include("../src/Nodosus.jl")

Main.Nodosus

Get access token,

In [4]:
@time access = Linckii.get_access(LinckiiSecrets.keab)

  0.001670 seconds (410 allocations: 27.828 KiB)


(url = "https://canary.noda.se/~keab", key = "e86d93131e6bc12ae701d036e696cf8b")

Load site,

In [5]:
@time nodes, devices, sensors = Linckii.loadsite(access, :nodes, :devices, :sensors)

 23.117180 seconds (20.99 M allocations: 1.373 GiB, 2.34% gc time)


(nodes = Table with 279 rows, 8 columns:
Columns:
#  colname           type
─────────────────────────────────
1  node_public       Bool
2  node_name         String
3  device_id         Int64
4  node_id           Int64
5  node_owner        Bool
6  node_enabled      Bool
7  sensor_ids        Array{Any,1}
8  node_description  String, devices = Table with 11 rows, 3 columns:
protocol_id  device_name             device_id
──────────────────────────────────────────────
1            "Kelp-IQHeat"           1
2            "Scout"                 2
1            "Kelp-Basic"            4
5            "Weather-Forecast"      6
6            "Grid-Basic"            7
6            "Grid-Controller"       8
7            "Heat-Load-Forecast"    9
2            "CMa1Xw"                10
2            "EcoGuard Sensor"       11
2            "WEPTech"               12
8            "Flexibility-Forecast"  13, sensors = Table with 175 rows, 5 columns:
Columns:
#  colname             type
───────────────────

Get side 1 substation series,

In [6]:
sensor_names = (
    :meter_effect,
    :meter_primreturntemp,
    :meter_primsupplytemp,
    :meter_volumeflow,
)

(:meter_effect, :meter_primreturntemp, :meter_primsupplytemp, :meter_volumeflow)

In [7]:
signals = JuliaDB.reindex(
    JuliaDB.join(
        Linckii.flatten_nodes(JuliaDB.filter(r -> r.device_id == 4, nodes)),
        JuliaDB.filter(r -> r.sensor_name in sensor_names, sensors);
        lkey = :sensor_id,
        rkey = :sensor_id,
    ),
    (:node_id, :sensor_id),
)

Table with 361 rows, 12 columns:
Columns:
#   colname             type
──────────────────────────────
1   node_id             Int64
2   sensor_id           Int64
3   node_public         Bool
4   node_name           String
5   device_id           Int64
6   node_owner          Bool
7   node_enabled        Bool
8   node_description    String
9   sensor_unit         Any
10  sensor_name         Symbol
11  protocol_id         Int64
12  sensor_description  String

In [8]:
signal_rows = JuliaDB.rows(signals)
length(signal_rows)

361

Load data,

In [9]:
p, q = Dates.Minute(10), Dates.Minute(120)

(10 minutes, 120 minutes)

In [19]:
r = signal_rows[Random.rand(1 : end)]
(r.node_id, r.sensor_name)

(56, :meter_primreturntemp)

In [24]:
filepath = "$(Linckii.datapath(access, r.node_id, r.sensor_name)).db"
data = Linckii.loaddata(access, r.node_id, r.sensor_name)
rows = JuliaDB.rows(data)
t = JuliaDB.transform(
    JuliaDB.table(Nodosus.pp1(rows, p, q); pkey = JuliaDB.pkeynames(data)),
    # vore mycket enklare om vi redan hade node_id som kolumn och primärnyckel
    :node_id => r.node_id,
    :value => r.sensor_name,
)

ErrorException: column 56 not found.

In [18]:
ppts = []
@time for r in signal_rows
    filepath = "$(Linckii.datapath(access, r.node_id, r.sensor_name)).db"
    try
        data = Linckii.loaddata(access, r.node_id, r.sensor_name)
        rows = JuliaDB.rows(data)
        push!(
            ppts,
            JuliaDB.transform(
                JuliaDB.table(Nodosus.pp1(rows, p, q); pkey = (:node_id, :datetime)),
                :node_id => r.node_id,
                :value => r.sensor_name,
            ),
        )
        println("$(filepath): Done")
    catch e
        println("$(filepath): $(e)")
    end
end
data = reduce((l, r) -> JuliaDB.join(l, r; how = :outer), ppts)

db/linckii/~keab/data/1/meter_effect.db: MethodError(isless, (600000 milliseconds, Main.Nodosus.var"#19#23"(Core.Box(Main.Nodosus.var"#19#23"(#= circular reference @-2 =#)))), 0x000000000000693b)
db/linckii/~keab/data/1/meter_primreturntemp.db: MethodError(isless, (600000 milliseconds, Main.Nodosus.var"#19#23"(Core.Box(Main.Nodosus.var"#19#23"(#= circular reference @-2 =#)))), 0x000000000000693b)
db/linckii/~keab/data/1/meter_primsupplytemp.db: MethodError(isless, (600000 milliseconds, Main.Nodosus.var"#19#23"(Core.Box(Main.Nodosus.var"#19#23"(#= circular reference @-2 =#)))), 0x000000000000693b)
db/linckii/~keab/data/1/meter_volumeflow.db: ErrorException("db/linckii/~keab/data/1/meter_volumeflow.db is not a file or directory")
db/linckii/~keab/data/16/meter_effect.db: ErrorException("db/linckii/~keab/data/16/meter_effect.db is not a file or directory")
db/linckii/~keab/data/16/meter_primreturntemp.db: ErrorException("db/linckii/~keab/data/16/meter_primreturntemp.db is not a file or di

ArgumentError: ArgumentError: reducing over an empty collection is not allowed

Save data,

In [11]:
@time JuliaDB.save(
    data,
    "$(Linckii.datapath(access)).db",
)

UndefVarError: UndefVarError: data not defined

Load data,

In [12]:
@time data = JuliaDB.load(
    "$(Linckii.datapath(access)).db",
)

ErrorException: db/linckii/~keab/data.db is not a file or directory

In [13]:
# two plots, one for a random node but both supply and return temp, and one for all nodes but only their difference

Complex plot,

In [14]:
t = JuliaDB.filter(
    r -> r.datetime >= Dates.DateTime(2018) && r.datetime <= Dates.DateTime(2019),
    data,
)
x = JuliaDB.select(t, :datetime)
y = JuliaDB.select(t, :meter_primreturntemp)
Plots.plot(x, y, xlabel = :datetime, label = Linckii.datapath(access, r.node_id, r.sensor_name))

UndefVarError: UndefVarError: data not defined